<a href="https://colab.research.google.com/github/ethandavenport/Optimization-I-Project-3/blob/main/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install gurobipy

In [9]:
# Load all required libraries
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.utils import resample  # for bootstrapping
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize


In [10]:
url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-3/main/price_demand_data.csv"
df = pd.read_csv(url)

In [11]:
df.head()

,price,demand
0,1.05,283
1,0.86,771
2,1.21,185
3,0.94,531
4,0.76,1002


# Part 6

In [12]:
df_boot = resample(df, replace=True, n_samples=len(df))
df_boot.head()

,price,demand
54,1.08,336
54,1.08,336
33,1.08,522
15,0.97,665
77,0.94,750


In [13]:
X = df_boot[["price"]].to_numpy()
y = df_boot["demand"].to_numpy()

ols = LinearRegression()
ols.fit(X, y)

a = float(ols.intercept_)
b = float(ols.coef_[0])
r2 = float(ols.score(X, y))

residuals = y - (a + b*X[:,0])

print(f"intercept: {a:.6f}")
print(f"slope: {b:.6f}")
print(f"r squared: {r2:.6f}")

intercept: 1949.835843
slope: -1364.061239
r squared: 0.674642


In [14]:
c = 0.5
g = 0.75
t = 0.15
p_fixed = 1.0

d1 = a + b*p_fixed + residuals # demand vector at p=1
n = d1.size
print("Scenarios at p=1:", n, "points")

Scenarios at p=1: 99 points


In [15]:
#model
mb = gp.Model()

p = mb.addMVar(1, lb=0.0, name="p") # price
q = mb.addMVar(1, lb=0.0, name="q") # quantity

s = mb.addMVar(n, lb=0.0, name="s") # sales
r = mb.addMVar(n, lb=0.0, name="r") # rush
d = mb.addMVar(n, lb=0.0, name="d") # disposal

Restricted license - for non-production use only - expires 2027-11-29


In [16]:
#constraints
d3 = a + b * p[0] + residuals # demand vector under decision p
mb.addConstr(s <= q[0])
mb.addConstr(s <= d3)
mb.addConstr(r >= d3 - q[0])
mb.addConstr(d >= q[0] - d3)

<MConstr (99,) *awaiting model update*>

In [17]:
#objective and solve
obj_boot = (p[0] * gp.quicksum(s) - c * n * q[0] - g * gp.quicksum(r) - t * gp.quicksum(d)) / n
mb.setObjective(obj_boot, GRB.MAXIMIZE)
mb.Params.OutputFlag = 0
mb.optimize()

p_star = float(p.X[0])
q_star_joint = float(q.X[0])
profit_joint = float(mb.objVal)

GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
print(f"optimal price: {p_star:.6f}")
print(f"optimal quantity: {q_star_joint:.6f}")
print(f"Expected profit: {profit_joint:.6f}")